# This notebook scans all directories in an input bucket and converts all files to wav

This occurs inplace and deletes the original file and replaces it with a .wav file, It overwrites an existing file by the same name.

It expects input of a "bucket" folder, consisting of subfolders named as species, and the audio files within.

Audio file inputs are
- .mp3
- .flac

Video file inputs are
- .mp4
- .m4a

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import librosa, librosa.display
import pathlib
import glob
import pathlib
import subprocess

In [ ]:
# input the bucket directory here
bucket ="/Users/mhallid2/ProjectEcho_Data/test_bucket orig"
species_bucket = os.listdir(bucket)
species_bucket

In [ ]:
# writes creates a dataframe of species (folder names)

species_bucket = pd.DataFrame(species_bucket, columns = ["animal species"])
species_bucket.index = np.arange(1, len(species_bucket) + 1)
species_bucket = species_bucket.sort_values("animal species")
species_bucket = species_bucket.iloc[1:]
species_bucket.index = np.arange(1, len(species_bucket) + 1)
print("files: {}".format(len(species_bucket)))
species_bucket.head()

In [ ]:
## extract species as dataframe
# function to crearte a dataframe of all species (folders) in the bucket, listed alphabetically
def species_as_dataframe(folder_filepath):
    species_list = os.listdir(folder_filepath)
    species_dataframe = pd.DataFrame(species_list, columns = ["animal species"])
    species_dataframe.index = np.arange(1, len(species_dataframe) + 1)
    species_dataframe = species_dataframe.sort_values("animal species")
    species_dataframe = species_dataframe.iloc[1:]
    species_dataframe.index = np.arange(1, len(species_dataframe) + 1)
    return species_dataframe 

species_as_dataframe(bucket)


In [ ]:
# function to crearte a dataframe of full directory names of species in the bucket, listed alphabetically

def file_paths_as_df(bucket):  # maybe change to receive the column value too
    bucket_folders = glob.glob(bucket+"/*")
    bucket_folders = pd.DataFrame(bucket_folders, columns = ["animal species folder paths"])
    bucket_folders = bucket_folders.sort_values("animal species folder paths")
    bucket_folders.index = np.arange(1, len(bucket_folders) + 1)
    bucket_folders.head()
    return bucket_folders
bucket_file_paths_as_dataframe = file_paths_as_df(bucket)
bucket_file_paths_as_dataframe

# start with a df and later change to list
### Function to list the filepaths of all species in that bucket as a list

In [ ]:
# this takes a dataframe and changes it back to list
def file_paths_as_list(bucket_folders_dataframe):
    bucket_folders_list = bucket_folders_dataframe.values.tolist()
    bucket_folders_list = list(np.concatenate(bucket_folders_list).flat)
    bucket_folders_list[0:1]
    return bucket_folders_list

In [ ]:
bucket_file_paths_as_list = file_paths_as_list(file_paths_as_df(bucket))
bucket_file_paths_as_list

### Function to list the filepaths of all species in that bucket as a dataframe

In [ ]:
# this creates a .wav version of a file past to it
# it will save with the same name
# it will overwrite if an existing .wav file exists with the same name

def convert_individual_file(file_path):
    parent_path = pathlib.PurePath(file_path).parent    
    file_name = pathlib.Path(file_path).stem   
    output_file_path = str(parent_path) + "/" + file_name + ".wav"
    # write the new wav file forcing overwrite 
    subprocess.call(['ffmpeg', '-y', '-i', file_path,
                 output_file_path])
    return

In [ ]:
# this function checks the filepath passed to it and checks the filetype
# if it is one of the listed filetypes ot be converted, it will make a new version 
# as a .wav and delete the original

def check_for_conversion(i, species_filepath):  
    audio_file_paths = glob.glob(species_filepath+"/*")

    for i in range(len(audio_file_paths)):
        if pathlib.Path(audio_file_paths[i]).suffix == ".mp3":
            convert_individual_file(audio_file_paths[i]) # save a new copy as wav in this directory
            os.remove(audio_file_paths[i])

        elif pathlib.Path(audio_file_paths[i]).suffix == ".flac":
            convert_individual_file(audio_file_paths[i]) # save a new copy as wav in this directory
            os.remove(audio_file_paths[i])

        elif pathlib.Path(audio_file_paths[i]).suffix == ".m4a":
            convert_individual_file(audio_file_paths[i]) # save a new copy as wav in this directory
            os.remove(audio_file_paths[i])

        elif pathlib.Path(audio_file_paths[i]).suffix == ".mp4":
            convert_individual_file(audio_file_paths[i]) # save a new copy as wav in this directory
            os.remove(audio_file_paths[i])
    return

In [ ]:
### working here ###

'''
pathlib.PurePath would work much better here I'm sure ...
'''

def convert_files(bucket_file_path):
    print("Folders scanned")
    print(".........")
    for i in range(len(species_bucket)-1):
        bucket_species_dataframe = species_as_dataframe(bucket_file_path)
        bucket_file_paths_as_list = file_paths_as_list(file_paths_as_df(bucket_file_path))
        current_species_file_pathame = bucket_file_paths_as_list[i]
        check_for_conversion(i, current_species_file_pathame)
    print(bucket_species_dataframe)
    print(".........")
    print("Conversion Complete")




In [ ]:
convert_files(bucket)

In [ ]:
# run a second time to generate a clean output
convert_files(bucket)